## Lab 9c: Markov Decision Processes, Value and Policy Iteration
Fill me in

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5760-Labs-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_wor

### Types

```julia
mutable struct MyRectangularGridWorldModel <: AbstractWorldModel

    # data -
    number_of_rows::Int
    number_of_cols::Int
    coordinates::Dict{Int,Tuple{Int,Int}}
    states::Dict{Tuple{Int,Int},Int}
    moves::Dict{Int,Tuple{Int,Int}}
    rewards::Dict{Int,Float64}

    # constructor -
    MyRectangularGridWorldModel() = new();
end
```

In [2]:
# Step 1: Build a model of the world -
number_of_rows = 5
number_of_cols = 5
nactions = 4; 
nstates = (number_of_rows*number_of_cols);
γ = 0.95;

# setup rewards -
rewards = Dict{Tuple{Int,Int}, Float64}()
rewards[(2,2)] = -100000.0 # lava is the (2,2) square 
rewards[(3,3)] = 1000.0    # target square

# setup set of absorbing states -
absorbing_state_set = Set{Tuple{Int,Int}}()
push!(absorbing_state_set, (2,2));
push!(absorbing_state_set, (3,3));

# call the factory -
world = VLDecisionsPackage.build(MyRectangularGridWorldModel, 
    (nrows = number_of_rows, ncols = number_of_cols, rewards = rewards));

In [3]:
world.moves

Dict{Int64, Tuple{Int64, Int64}} with 4 entries:
  4 => (0, 1)
  2 => (1, 0)
  3 => (0, -1)
  1 => (-1, 0)

In [4]:
world.states

Dict{Tuple{Int64, Int64}, Int64} with 25 entries:
  (4, 5) => 20
  (1, 2) => 2
  (3, 1) => 11
  (2, 5) => 10
  (1, 3) => 3
  (1, 4) => 4
  (5, 5) => 25
  (3, 2) => 12
  (3, 3) => 13
  (4, 1) => 16
  (2, 1) => 6
  (3, 4) => 14
  (1, 5) => 5
  (4, 2) => 17
  (5, 1) => 21
  (2, 2) => 7
  (4, 3) => 18
  (2, 3) => 8
  (3, 5) => 15
  (4, 4) => 19
  (2, 4) => 9
  (1, 1) => 1
  (5, 2) => 22
  (5, 3) => 23
  (5, 4) => 24

In [5]:
𝒮 = range(1,stop=nstates,step=1) |> collect;
𝒜 = range(1,stop=nactions,step=1) |> collect;

In [6]:
R = zeros(nstates, nactions);
fill!(R, 0.0)
for s ∈ 𝒮
    for a ∈ 𝒜
        
        Δ = world.moves[a];
        current_position = world.coordinates[s]
        new_position =  current_position .+ Δ
        if (haskey(world.states, new_position) == true)
            if (haskey(rewards, new_position) == true)
                R[s,a] = rewards[new_position];
            else
                R[s,a] = -1.0;
            end
        else
            R[s,a] = -50000.0; # we are off the grid, big negative penalty
        end
    end
end
R

25×4 Matrix{Float64}:
  -50000.0       -1.0   -50000.0       -1.0
  -50000.0  -100000.0       -1.0       -1.0
  -50000.0       -1.0       -1.0       -1.0
  -50000.0       -1.0       -1.0       -1.0
  -50000.0       -1.0       -1.0   -50000.0
      -1.0       -1.0   -50000.0  -100000.0
      -1.0       -1.0       -1.0       -1.0
      -1.0     1000.0  -100000.0       -1.0
      -1.0       -1.0       -1.0       -1.0
      -1.0       -1.0       -1.0   -50000.0
      -1.0       -1.0   -50000.0       -1.0
 -100000.0       -1.0       -1.0     1000.0
      -1.0       -1.0       -1.0       -1.0
      -1.0       -1.0     1000.0       -1.0
      -1.0       -1.0       -1.0   -50000.0
      -1.0       -1.0   -50000.0       -1.0
      -1.0       -1.0       -1.0       -1.0
    1000.0       -1.0       -1.0       -1.0
      -1.0       -1.0       -1.0       -1.0
      -1.0       -1.0       -1.0   -50000.0
      -1.0   -50000.0   -50000.0       -1.0
      -1.0   -50000.0       -1.0       -1.0
      -1.0

In [7]:
T = Array{Float64,3}(undef, nstates, nstates, nactions);
fill!(T, 0.0)
for i ∈ 1:nactions
    
    aᵢ = 𝒜[i];
    Δ = world.moves[aᵢ];
    
    for s ∈ 𝒮
        current_position = world.coordinates[s]
        new_position =  current_position .+ Δ
        if (haskey(world.states, new_position) == true && 
                in(current_position, absorbing_state_set) == false)
            s′ = world.states[new_position];
            T[s, s′,  aᵢ] = 1.0
        else
            T[s, s,  aᵢ] = 1.0
        end
    end
end

In [15]:
T[7,1:end,1]

25-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [9]:
m = VLDecisionsPackage.build(MyMDPProblemModel, 
    (𝒮 = 𝒮, 𝒜 = 𝒜, T = T, R = R, γ = γ));

In [10]:
value_iteration_model = MyValueIterationModel(100);

In [11]:
test = VLDecisionsPackage.solve(value_iteration_model, m)

MyValueFunctionPolicy(MyMDPProblemModel([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [1, 2, 3, 4], [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0;;; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0;;; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 1.0], [-50000.0 -1.0 -50000.0 -1.0; -50000.0 -100000.0 -1.0 -1.0; … ; -1.0 -50000.0 -1.0 -1.0; -1.0 -50000.0 -1.0 -50000.0], 0.95), [838.3507855844066, 883.5209105844066, 931.0684105844067, 883.5209105844066, 838.3507855844066, 883.5209105844066, -19.8815894155933, 981.1184105844067, 931.0684105844067, 883.5209105844066  …  883.5209105844066, 931.0684105844067, 981.1184105844067, 931.0684105844067, 883.5209105844066, 838.3507855844066, 883.5209105844066, 931.0684105844067, 883.5209105844066, 838.3507855844066])

In [12]:
my_π = Q(m, test.U) |> policy

25-element Vector{Int64}:
 2
 4
 2
 2
 2
 2
 1
 2
 2
 2
 4
 4
 1
 3
 3
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [13]:
move_arrows = Dict{Int,Any}();
move_arrows[1] = "←"
move_arrows[2] = "→"
move_arrows[3] = "↓"
move_arrows[4] = "↑"
move_arrows[5] = "∅"

"∅"

In [14]:
for s ∈ 𝒮
    a = my_π[s];
    Δ = world.moves[a];
    current_position = world.coordinates[s]
    new_position =  current_position .+ Δ
    
    if (in(current_position, absorbing_state_set) == true)
        println("$(current_position) $(move_arrows[5])")
    else
        println("$(current_position) $(move_arrows[a]) $(new_position)")
    end
end

(1, 1) → (2, 1)
(1, 2) ↑ (1, 3)
(1, 3) → (2, 3)
(1, 4) → (2, 4)
(1, 5) → (2, 5)
(2, 1) → (3, 1)
(2, 2) ∅
(2, 3) → (3, 3)
(2, 4) → (3, 4)
(2, 5) → (3, 5)
(3, 1) ↑ (3, 2)
(3, 2) ↑ (3, 3)
(3, 3) ∅
(3, 4) ↓ (3, 3)
(3, 5) ↓ (3, 4)
(4, 1) ← (3, 1)
(4, 2) ← (3, 2)
(4, 3) ← (3, 3)
(4, 4) ← (3, 4)
(4, 5) ← (3, 5)
(5, 1) ← (4, 1)
(5, 2) ← (4, 2)
(5, 3) ← (4, 3)
(5, 4) ← (4, 4)
(5, 5) ← (4, 5)
